<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tinysegmenter

  Created wheel for tinysegmenter: filename=tinysegmenter-0.4-cp36-none-any.whl size=13536 sha256=e88266293c8aaad6ef234a425ac8fa1b42f944b14152b056bb658a3546d1a026
  Stored in directory: /root/.cache/pip/wheels/68/71/2b/6402196bf28012826e507ef7b99df6ebd98cce78bd99023471
Successfully built tinysegmenter


*Kurohashi-Kawahara Lab. has the copyright of Japanese Basic Sentence Data, and NICT MASTAR Project, Multilingual Translation Lab. has the copyright of English and Chinese Basic Sentence Data. You can use all the data under the terms of the Creative Commons Attribution 3.0 Unported license.
     http://nlp.ist.i.kyoto-u.ac.jp/EN/?JEC%20Basic%20Sentence%20Data*

In [2]:
import math
import numpy as np
import pandas as pd
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from matplotlib import pyplot as plt


from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import tinysegmenter

In [3]:
segmenter = tinysegmenter.TinySegmenter()

In [4]:
device = torch.device('cuda:0')

In [5]:
my_frame = pd.read_excel('http://nlp.ist.i.kyoto-u.ac.jp/EN/?plugin=attach&refer=JEC%20Basic%20Sentence%20Data&openfile=JEC_basic_sentence_v1-2.xls')

In [6]:
#remove Chineese column
my_frame = my_frame.drop(['难道不会是X吗，我实在是感到怀疑。'], axis=1)
my_frame.columns = ['index', 'jap', 'eng']

In [7]:
my_frame

,index,jap,eng
0,#0002,Xがいいなといつも思います,I always think X would be nice.
1,#0003,それがあるようにいつも思います,It always seems like it is there.
2,#0004,それが多すぎないかと正直思う,I honestly feel like there is too much.
3,#0005,山田はみんなに好かれるタイプの人だと思う,I think that Yamada is the type everybody likes.
4,#0006,〜と誰かが思った,Someone thought that 〜
...,...,...,...
5298,#5300,チームが４人のメンバーで構成されています,The team consists of four members.
5299,#5301,彼が実際に動画を再生する,He actually plays the video.
5300,#5302,政府が銀行に公的資金をどんどん投入しました,The government injected massive public funds i...
5301,#5303,レベル１の機能に下記の機能をプラスする,The following will be added to the level 1 fun...


In [8]:
segmenter.tokenize(my_frame['jap'][0])

['X', 'が', 'いい', 'な', 'といつも', '思い', 'ます']

In [9]:
# pairs = [list()]

In [10]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    # s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [12]:
ppp = [[11],[33]]
ppp = list(reversed(ppp))
ppp

[[33], [11]]

In [13]:
    for index, sent in enumerate(my_frame['jap']):
      if index == 1:
        print(index, sent)
      pair = [normalizeString(my_frame['eng'][index]), ' '.join(segmenter.tokenize(sent))]
      if index == 1:
        print(pair)

1 それがあるようにいつも思います
['It always seems like it is there .', 'それ が ある よう にいつも 思い ます']


In [14]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    # lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    #     read().strip().split('\n')
    pairs = []
    # Split every line into pairs and normalize
    for index, sent in enumerate(my_frame['jap']):
      pair = [normalizeString(my_frame['eng'][index]), ' '.join(segmenter.tokenize(sent))]
      pairs.append(pair)

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [15]:
MAX_LENGTH = 20

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
        # p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'jap', False)
print(random.choice(pairs))

Reading lines...
Read 5303 sentence pairs
Trimmed to 5209 sentence pairs
Counting words...
Counted words:
eng 5753
jap 6963
['I did a live show with him at a club .', '彼 と 一緒 に ライブハウス で ライブ を やっ た']


In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [19]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [21]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

__main__.AttnDecoderRNN

In [22]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [23]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [24]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [25]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [26]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [31]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [32]:
input_lang.n_words

5753

In [37]:
evaluate(encoder1, attn_decoder1, "I want to kill")

(['流れ',
  '構成',
  '横道',
  '遭う',
  '勧め',
  '世紀',
  '大正',
  '短期',
  '摂り',
  '向こう',
  '全',
  '世紀',
  '大正',
  '短期',
  '摂り',
  '向こう',
  '全',
  '世紀',
  '世紀',
  '使わ'],
 tensor([[0.1022, 0.0646, 0.0436, 0.0425, 0.0347, 0.0374, 0.0311, 0.0488, 0.0547,
          0.0463, 0.0558, 0.0339, 0.0490, 0.0560, 0.0326, 0.0555, 0.0666, 0.0475,
          0.0551, 0.0421],
         [0.0174, 0.0647, 0.0576, 0.0383, 0.0302, 0.0422, 0.0413, 0.0252, 0.1138,
          0.0530, 0.0828, 0.0410, 0.0429, 0.0632, 0.0361, 0.0413, 0.0281, 0.0354,
          0.0611, 0.0844],
         [0.0317, 0.0618, 0.0390, 0.0262, 0.0515, 0.0669, 0.0440, 0.0762, 0.0235,
          0.0530, 0.0401, 0.0428, 0.0583, 0.0519, 0.0357, 0.0548, 0.0445, 0.0312,
          0.0780, 0.0889],
         [0.0599, 0.0982, 0.0802, 0.0350, 0.0433, 0.0324, 0.0607, 0.0485, 0.0293,
          0.0237, 0.0575, 0.0715, 0.0200, 0.0318, 0.0787, 0.0313, 0.0679, 0.0203,
          0.0181, 0.0917],
         [0.0887, 0.0488, 0.0708, 0.0493, 0.0693, 0.0353, 0.0497, 0.0184, 0

In [52]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

2m 19s (- 32m 27s) (5000 6%) 4.6875
4m 27s (- 29m 1s) (10000 13%) 4.2664
6m 35s (- 26m 21s) (15000 20%) 3.9971
8m 50s (- 24m 19s) (20000 26%) 3.7742
11m 0s (- 22m 1s) (25000 33%) 3.5385
13m 10s (- 19m 45s) (30000 40%) 3.2970
15m 19s (- 17m 31s) (35000 46%) 3.0348
17m 28s (- 15m 17s) (40000 53%) 2.7957
19m 38s (- 13m 5s) (45000 60%) 2.5653
21m 46s (- 10m 53s) (50000 66%) 2.4001
23m 56s (- 8m 42s) (55000 73%) 2.1450
26m 5s (- 6m 31s) (60000 80%) 2.0174
28m 15s (- 4m 20s) (65000 86%) 1.8899
30m 26s (- 2m 10s) (70000 93%) 1.7675
32m 37s (- 0m 0s) (75000 100%) 1.7408


In [53]:
evaluateRandomly(encoder1, attn_decoder1)

> It s easy to get people to understand when you are trying to protect their rights .
= 権利 を 守ろ う と する 方 が 、 周囲 の 理解 を 得られる
< 権利 する 者 が 守ろ する する う と する う う と う <EOS>

> We moved to our current house days after his birth .
= 彼 が 生後 ４ ２ 日 目 に 今 の 家 に 引っ越す
< 彼 が が ３ 、 彼 の 、 に か 思わず <EOS>

> I overdo it .
= 自分 が 無理 を する
< 自分 が 書い て 書い た <EOS>

> The number of people increases year by year .
= 年 々人 の 数 が 増える
< 年 が 数 が 増える <EOS>

> He made an obscene remark .
= 彼 が アダルト 的 発言 を し まし た
< 彼 が 水玉 を 人 を し ます <EOS>

> The following will be added to the level functions 
= レベル １ の 機能 に 下記 の 機能 を プラス する
< 下記 者 が の 機能 の を を ます <EOS>

> Someone said something like that .
= そんな事 を 誰か が 口 に し た
< その ほう が 、 そんな に し た <EOS>

> X do does not come into the view at all .
= X が 視界 に 全く 入り ませ ん
< X が 全く 全く 全く は に を <EOS>

> In that case you need to go through the re routing procedure .
= その 場合 に は ルート 変更 の 手続き が 必要 に なり ます
< その 場合 の 変更 に は 、 が が ます <EOS>

> He shows children how hard he tries .
= 彼 が 最大 限 の 努力 

In [40]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShow(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))



evaluateAndShow("I want to eat")
evaluateAndShow("I want to kill")
evaluateAndShow("I want to drink")
evaluateAndShow("I kill you")

input = I want to eat
output = 分解 組織 必ず多く みじん 蓄え 使う方 分かり 摂り お互い ほんとう 禁錮 必ず多く 禁錮 必ず多く 勧め がみんな 車窓 超え 瞑る 瞑る
input = I want to kill
output = 言わん 忙しく 過去 曲 被告 禁錮 必ず多く 横道 横道 禁錮 必ず多く 禁錮 必ず多く 勧め 過去 被告 今年 禁錮 必ず多く 横道
input = I want to drink
output = 店頭 全面 留学 禁錮 禁錮 図られ 図られ かける 短期 市販 実 新 全 禁錮 必ず多く 出力 禁錮 必ず多く 横道 横道
input = I kill you
output = 流れ パスタ 留学 潜ん 今年 筆者 禁錮 必ず多く 勧め 新 被告 禁錮 図られ 被告 短期 励まし 全 留学 禁錮 大正
